In [28]:
import glob

ignoreFiles= ["tags.md", "index.md","PLACEHOLDER.md", "GlobalFunctions.md", "Globals.md", "ModReference.md"]

docClasses = {}

for filepath in glob.iglob('../../docs/*.md'):
    trimFileName = filepath.split("\\")[len(filepath.split("\\"))-1]
    if trimFileName in ignoreFiles:
        continue

    className = trimFileName.replace(".md","")
    docClasses[className] = []

    file = open(filepath, 'r')
    docLines = file.readlines()
    for line in docLines:
        if line.strip().startswith("###") and not line.strip().startswith("####") and "aria-label" in line:
            filteredLine = line.replace("#","").replace("Â·","").replace("()","").split("{")[0].strip()
            docClasses[className].append(filteredLine)


In [29]:
file2 = open('api_dump.dat', 'r')
bindsLines = file2.readlines()

nameReplace = {
    "APIOverride.OverriddenClasses.Level.Original.__const": "Level",
    "APIOverride.OverriddenClasses.Level.Original": "Level",
    "ItemConfig.Config": "ItemConfig"
}

classes = {}

curClass= ""
curClassLVL= 0
for line in bindsLines:
    if not line.strip():
        continue
    lStripLine= line.lstrip()
    leadingTabs = len(line) - len(lStripLine)
    if lStripLine.startswith("Class") or lStripLine.startswith("Namespace") :
        className = lStripLine.split(" ")[1]
        if className in nameReplace:
            className = nameReplace[className]
        className = className.replace(".","_").strip() # make it doc complient
        classes[className] = []
        curClass = className
        curClassLVL = leadingTabs
    elif lStripLine.startswith("Table") or lStripLine.startswith("Subclasses") :
        continue
    else:
        if curClass and curClassLVL < leadingTabs:
            filterFunc = line.strip().split(" ")[0].split(":")[0]
            if filterFunc == "__call":
                continue
            classes[curClass].append(filterFunc)

### Classes and functions that are not documented, but were found by the current automatic api grabber tool

In [30]:
for dumpClass in classes:
    if dumpClass not in docClasses:
        if len(classes[dumpClass])==0:
            continue
        print("NOT DOCUMENTED CLASS:", dumpClass)
    else:
        docClassContents = docClasses[dumpClass]
        dumpClassContents = classes[dumpClass]
        for dumpContent in dumpClassContents:
            if dumpContent not in docClassContents:
                print("NOT DOCUMENTED CONTENT IN CLASS:",dumpClass,"\t", "CONTENT:",dumpContent)

NOT DOCUMENTED CONTENT IN CLASS: EntityPlayer 	 CONTENT: SpriteScale
NOT DOCUMENTED CONTENT IN CLASS: Sprite 	 CONTENT: SetFrame_1


### Classes and functions that are documented, but not found by the current automatic api grabber tool

In [31]:
# Ignore these, because they are from the main.lua file of the game
ignoreFuncs = ["AddTearFlags","ClearTearFlags", "HasTearFlags", "Color.Default", "Vector.Zero", "Vector.One", "ClearProjectileFlags", "HasProjectileFlags"]
depricatedFuncs = ["EntityPlayer.SecondaryActiveItem", "EntityPlayer.SetTargetTrapDoor","EntityTear.Bounced","GridEntityPoop.ReducedSpawnRate","ItemConfig_Item.Discharged","Room.TryPlaceLadder","GridEntityRock.GetSprite"]

for docClass in docClasses:
    if docClass not in classes:
        print("NOT CAPTURED CLASS:", docClass)
    else:
        docClassContents = docClasses[docClass]
        dumpClassContents = classes[docClass]
        for content in docClassContents:
            if content not in dumpClassContents:
                if content == docClass or content in ignoreFuncs or docClass+"."+content in depricatedFuncs:
                    # Ignore Constructors
                    continue
                print("NOT CAPTURED CONTENT IN CLASS:",docClass,"\t", "CONTENT:",content)